In [1]:
!pip3 install kfp --upgrade

!pip3 install {USER_FLAG} google-cloud-aiplatform==1.7.0 --upgrade
!pip3 install {USER_FLAG} kfp==1.8.9 google-cloud-pipeline-components==0.2.0

In [20]:
USER_FLAG = "--user"
!pip3 install {USER_FLAG} google-cloud-aiplatform==1.7.0 --upgrade
!pip3 install {USER_FLAG} kfp==1.8.9 google-cloud-pipeline-components==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.0 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.3/296.3 kB 7.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 79.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 78.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 79.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━

In [1]:
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component
from kfp.v2.google.client import AIPlatformClient

In [2]:
PROJECT_ID = 'digital-proton-368805'
REGION = 'us-central1'
BUCKET_NAME = 'gs://vertext-ai-piepline'

In [3]:
PIPELINE_ROOT = "{}/pipeline_root2/".format(BUCKET_NAME)

In [4]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

env: PATH=/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


In [5]:
DISPLAY_NAME = 'image_classification_footware'

In [6]:
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

In [7]:
@dsl.pipeline(name="image-classification",
                  pipeline_root=PIPELINE_ROOT)
def pipeline(
    gcs_source: str = "gs://automl_image_dataset/Shoe_vs_Sandal_vs_Boot_Dataset-1/image-classify-dataset.csv",
    display_name: str = DISPLAY_NAME,
    project: str = PROJECT_ID,
    gcp_region: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    #First component
    dataset_create_op = gcc_aip.ImageDatasetCreateOp(project=project, 
                                                     display_name=display_name, 
                                                     gcs_source=gcs_source,
                                                     import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification)
    
    #Second component
    training_op = gcc_aip.AutoMLImageTrainingJobRunOp(
        project=project,
        display_name=display_name,
        prediction_type="classification",
        budget_milli_node_hours=8000,
        dataset=dataset_create_op.outputs["dataset"],
    )
   
    #Fourth component is end point creation only if the condition is met
    endpoint_op = gcc_aip.EndpointCreateOp(
            project=project,
            location=gcp_region,
            display_name="train-automl-vision",
    )
    
    #Fifth component of the pipelin is deploying model on the endpoint created. 
    gcc_aip.ModelDeployOp(
            model=training_op.outputs["model"],
            endpoint=endpoint_op.outputs["endpoint"],
            automatic_resources_min_replica_count=1,
            automatic_resources_max_replica_count=1,
    )

In [8]:
compiler.Compiler().compile(pipeline_func=pipeline, package_path="image_classif_pipeline.json")

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1266: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [9]:
ml_pipeline_job = aiplatform.PipelineJob(
    display_name="automl-tabular-training",
    template_path="image_classif_pipeline.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"project": PROJECT_ID, "display_name": DISPLAY_NAME},
    enable_caching=True
)

In [10]:
ml_pipeline_job.submit()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/611484207798/locations/us-central1/pipelineJobs/image-classification-20221206105317
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/611484207798/locations/us-central1/pipelineJobs/image-classification-20221206105317')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/image-classification-20221206105317?project=611484207798


In [12]:
@component(output_component_file='getdata.yaml', base_image='python:3.9')
def getdata():
    import pandas as pd
    data = pd.read_csv("gs://custom_container_training/IRIS.csv")
    return data

In [13]:
@component(output_component_file='train.yaml', base_image='python:3.9')
def train():
    from sklearn.model_selection import train_test_split
    array = data.values
    X = array[:,0:4]
    y = array[:,4]
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20, random_state=1)
    return X_train, X_test, y_train, y_test

In [8]:
@dsl.pipeline(name="my-first-pipeline2", pipeline_root=PIPELINE_ROOT) 
def ml_pipeline():
    result = getdata(a,b)
    multiply(result.output, b)

In [16]:
compiler.Compiler().compile(
pipeline_func=add_mul_pipeline, package_path="hw_pipeline_job.json"
)